# <a name="Part4_3">4.3. Word2Vec  Embedding</a>
# Feature Type : Word2Vec ; Model - Logistic Regression - Hyper parameter tuning

## 1. Import and analyse the data set.

In [ ]:
import pandas as pd # read data file, data processing
import numpy as np # linear algebra
import matplotlib.pyplot as plt # plotting graph for EDA , Metrics analysis
%matplotlib inline
import seaborn as sns # plotting graph for EDA , Metrics analysis

from sklearn.pipeline import Pipeline

### Load the data 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Install latest version of gensim
!pip install -U gensim

     |████████████████████████████████| 24.1 MB 1.4 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [ ]:
import sys
import os

py_file_location = "/content/drive/MyDrive/AIML/projects/Capstone-NLP-Ticketing/"
sys.path.append(os.path.abspath(py_file_location))



In [ ]:
# Input data files has been processed for 
# 1. carriage return characters like '_x000D_' and \n 
# 2. Accented encoding character like äº§å“æ‰€åœ¨ä»“åº“å‡ºé”™ã€ , è¿žæŽ¥åŽè‡ªåŠ¨æ–­å¼€ï¼Œæ
# 3. Translation of words in non english language especially German, Italian, French
# Above 3 steps are done separately and output from these steps are used for further processing in Part 2
# 4. Update of Assigment group - fewer data groups , grouped to Group others
# 5. Pre-process for having only English data after translation, removal of spaces 
# 6. Treatment of Null values
# Above step 4,5,6 are done in part2 and processed data is stored in input_data_trans_preprocess.csv

data_dir = "/content/drive/MyDrive/AIML/projects/Capstone-NLP-Ticketing/"
data_file_name='input_data_trans_preprocess.csv'
#data_file_name='input_data.xlsx'
data_file_path = data_dir+data_file_name
data_file_path

'/content/drive/MyDrive/AIML/projects/Capstone-NLP-Ticketing/input_data_trans_preprocess.csv'

In [ ]:
#df_data = pd.read_excel(data_file_path)
df_data = pd.read_csv(data_file_path)

In [ ]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8467 entries, 0 to 8466
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Short description       8467 non-null   object
 1   Description             8467 non-null   object
 2   Caller                  8467 non-null   object
 3   Assignment group        8467 non-null   object
 4   orig_desc               8466 non-null   object
 5   orig_short_desc         8459 non-null   object
 6   Lang                    8467 non-null   object
 7   Translated_ShortDesc    8450 non-null   object
 8   Translated_Description  8467 non-null   object
 9   orig_assign_group       8467 non-null   object
dtypes: object(10)
memory usage: 661.6+ KB


**Feature with both description - Merging both Description and Short description**

In [ ]:
df_data.head(5)

,Short description,Description,Caller,Assignment group,orig_desc,orig_short_desc,Lang,Translated_ShortDesc,Translated_Description,orig_assign_group
0,login issue,verified user details employee manager name ch...,spxjnwir pjlcoqds,GRP_0,-verified user details.(employee# & manager na...,login issue,en,login issue,-verified user details.(employee# & manager na...,GRP_0
1,outlook,received from hmjdrvpb komuaywn gmail com hell...,hmjdrvpb komuaywn,GRP_0,_x000D_\n_x000D_\nreceived from: hmjdrvpb.komu...,outlook,en,outlook,received from: hmjdrvpb.komuaywn@gmail.com...,GRP_0
2,cant log in to vpn,received from eylqgodm ybqkwiam gmail com hi i...,eylqgodm ybqkwiam,GRP_0,_x000D_\n_x000D_\nreceived from: eylqgodm.ybqk...,cant log in to vpn,en,cant log in to vpn,received from: eylqgodm.ybqkwiam@gmail.com...,GRP_0
3,unable to access hr tool page,unable to access hr tool page,xbkucsvz gcpydteq,GRP_0,unable to access hr_tool page,unable to access hr_tool page,en,unable to access hr_tool page,unable to access hr_tool page,GRP_0
4,skype error,skype error,owlgqjme qhcozdfx,GRP_0,skype error,skype error,no,skype error,skype error,GRP_0


## Word2Vec embedding

In [ ]:
# Create a new column by merging both description field and use this for model training and prediction
df_data['Desc_All'] = df_data['Short description'] + ' '+ df_data['Description']

In [ ]:
max_words = df_data['Desc_All'].apply(lambda x: len(x.split())).max()
min_words = df_data['Desc_All'].apply(lambda x: len(x.split())).min()
mean_words = df_data['Desc_All'].apply(lambda x: len(x.split())).mean()
print("Max words in a ticket description ", max_words)
print("Min words in a ticket description", min_words)
print("Avearage words in a ticket description  ", round(mean_words,2))

Max words in a ticket description  1587
Min words in a ticket description 2
Avearage words in a ticket description   35.69


In [ ]:
X = df_data['Desc_All']
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords # Import stop words

stop_words = set(stopwords.words('english'))
from nltk.stem import WordNetLemmatizer

lem=WordNetLemmatizer()
corpus=[]
for tickets in X:
  words=[w for w in word_tokenize(tickets) if (w not in stop_words)]
  words=[lem.lemmatize(w) for w in words if len(w)>2]
  corpus.append(words)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from nltk.corpus import wordnet
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
#1. STOPWORD REMOVAL
def stopword(string):
    a= [i for i in string.split() if i not in stopwords.words('english')]
    return ' '.join(a)

#3. LEMMATIZATION
# Initialize the lemmatizer
wl = WordNetLemmatizer()
 
# This is a helper function to map NTLK position tags
# Full list is available here: https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

# Tokenize the sentence
def lemmatizer(string):
    word_pos_tags = nltk.pos_tag(word_tokenize(string)) # Get position tags
    a=[wl.lemmatize(tag[0], get_wordnet_pos(tag[1])) for idx, tag in enumerate(word_pos_tags)] # Map the position tag and lemmatize the word/token
    return " ".join(a)

#FINAL PREPROCESSING
def finalpreprocess(string):
    return lemmatizer(stopword(string))

In [ ]:
# df_data = df_data[df_data['Assignment group'] != 'GRP_0'] # Condition to check how model behaves if Group 0 data is removed from train data
feature_name = 'Desc_All'
df_data['clean_text'] =  df_data[feature_name].apply(lambda x: finalpreprocess(x))


#### Building Word2Vec model

In [ ]:
#building Word2Vec model

from gensim.models import Word2Vec
# The size of the dense vector to represent each token or word (i.e. the context or neighboring words).
# If you have limited data, then size should be a much smaller value since you would only have so many unique neighbors for a given word.
#min_count=1 means word should be present at least across all documents,
#if min_count=5 means if the word is present less than 5 times across all the documents then we shouldn't consider it
#w2v_model = Word2Vec(corpus, vector_size=50, window=5, min_count=5, workers=4) 

df_data['clean_text_tok'] = [nltk.word_tokenize(i) for i in df_data['clean_text']] #convert preprocessed sentence to tokenized sentence

w2v_model = Word2Vec(df_data['clean_text_tok'], vector_size=10, window=5, min_count=10, workers=4) 

In [ ]:
len(w2v_model.wv.index_to_key)

1870

In [ ]:

#for converting sentence to vectors/numbers from word vectors result by Word2Vec
class MeanEmbeddingVectorizer(object):
  def __init__(self, word2vec):
      self.word2vec = word2vec
      # if a text is empty we should return a vector of zeros
      # with the same dimensionality as all the other vectors
      self.dim = len(next(iter(word2vec.values())))
  def fit(self, X, y):
          return self
  def transform(self, X):
        ## Create document vectors by averaging word vectors. Empty we should return a vector of zeros.
          return np.array([
              np.mean([self.word2vec[w] for w in words if w in self.word2vec]
                  or [np.zeros(self.dim)], 
                      axis=0)
              for words in X
          ])

#combination of word and its vector
w2v = dict(zip(w2v_model.wv.index_to_key, w2v_model.wv.key_to_index))
modelw = MeanEmbeddingVectorizer(w2v)


In [ ]:
from sklearn.model_selection import train_test_split

X = df_data['clean_text_tok']
y = df_data['Assignment group'].values
X_train, X_prod, y_train, y_prod = train_test_split(X, y, test_size=0.05, random_state=0, stratify=y)
print('Prod Shape', X_prod.shape )
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=0, stratify=y)
print('Train shape', len(X_train))
print('Val shape', len(X_val))


Prod Shape (424,)
Train shape 6350
Val shape 2117


In [ ]:
# Convert Target variable to categorical value using label encoding

from sklearn import preprocessing
from tensorflow.keras.utils import to_categorical

y = df_data['Assignment group'].values
le = preprocessing.LabelEncoder()
le.fit(y)
num_classes = len(le.classes_)
y_train_mdl_lbl_enc = le.transform(y_train)
y_train_mdl_cat = to_categorical(y_train_mdl_lbl_enc, num_classes)
y_val_mdl_lbl_enc = le.transform(y_val)
y_val_mdl_cat = to_categorical(y_val_mdl_lbl_enc, num_classes)

In [ ]:
print(X_train)

7660       [unable, load, outlook, unable, load, outlook]
3477    [urgent, help, require, crm, mobile, app, load...
3734    [telephone, defect, gigaset, extension, batter...
7278    [connect, vpn, name, pfzxecbo, ptygkvzl, langu...
5979                 [login, help, hub, login, help, hub]
                              ...                        
2525    [hardware, item, come, wrong, labeling, wareho...
3338    [job, job, fail, job, scheduler, receive, moni...
7169    [unable, change, quotation, unable, change, qu...
4979    [unable, sign, skype, confirm, email, address,...
5788         [unable, connect, vpn, unable, connect, vpn]
Name: clean_text_tok, Length: 6350, dtype: object


In [ ]:
#Word2Vec
# Word2Vec runs on tokenized sentences
X_train_tok= [nltk.word_tokenize(str(i)) for i in X_train]  
X_val_tok= [nltk.word_tokenize(str(i)) for i in X_val]

In [ ]:
X_train_vectors_w2v = modelw.transform(X_train_tok)
X_val_vectors_w2v = modelw.transform(X_val_tok)

In [ ]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression

clf_LR = OneVsRestClassifier(LogisticRegression(max_iter=250))


In [ ]:
clf_LR.fit(X_train_vectors_w2v,y_train_mdl_lbl_enc)
predictions = clf_LR.predict(X_val_vectors_w2v)


In [ ]:
from sklearn.metrics import accuracy_score , f1_score, precision_score, recall_score
from sklearn.metrics import classification_report

def calc_metrics(actual,predicted):
  print('Accuracy score: ', round(accuracy_score(actual, predicted),2))
  print("precision_weighted:", round(precision_score(actual, predicted,average='weighted', zero_division=1),2))
  print("recall_weighted:", round(recall_score(actual, predicted,average='weighted', zero_division=1 ),2))
  print("f1_weighted:", round(f1_score(actual, predicted,average='weighted', zero_division=1 ),2))
  print("Classification Report:")
  print(classification_report(actual, predicted,zero_division=1))

In [ ]:
calc_metrics(y_val_mdl_lbl_enc,predictions)

Accuracy score:  0.47
precision_weighted: 0.75
recall_weighted: 0.47
f1_weighted: 0.3
Classification Report:
              precision    recall  f1-score   support

           0       0.47      1.00      0.64       994
           1       1.00      0.00      0.00         8
           2       1.00      0.00      0.00        35
           3       1.00      0.00      0.00         8
           4       1.00      0.00      0.00        64
           5       1.00      0.00      0.00        36
           6       1.00      0.00      0.00        30
           7       1.00      0.00      0.00        10
           8       1.00      0.00      0.00        21
           9       1.00      0.00      0.00        20
          10       1.00      0.00      0.00        22
          11       1.00      0.00      0.00        54
          12       1.00      0.00      0.00        60
          13       1.00      0.00      0.00         9
          14       1.00      0.00      0.00         7
          15       1.00   

#### Hyper Parameter tuning Word2Vec Model

In [ ]:


from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression

LR_pipeline_hyper_tuning = Pipeline( steps = [ 
                            ('w2v',MeanEmbeddingVectorizer(w2v)),                            
                            ('clf_LR', OneVsRestClassifier(LogisticRegression()))
                            ])



In [ ]:
LR_pipeline_hyper_tuning.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'w2v', 'clf_LR', 'clf_LR__estimator__C', 'clf_LR__estimator__class_weight', 'clf_LR__estimator__dual', 'clf_LR__estimator__fit_intercept', 'clf_LR__estimator__intercept_scaling', 'clf_LR__estimator__l1_ratio', 'clf_LR__estimator__max_iter', 'clf_LR__estimator__multi_class', 'clf_LR__estimator__n_jobs', 'clf_LR__estimator__penalty', 'clf_LR__estimator__random_state', 'clf_LR__estimator__solver', 'clf_LR__estimator__tol', 'clf_LR__estimator__verbose', 'clf_LR__estimator__warm_start', 'clf_LR__estimator', 'clf_LR__n_jobs'])

In [ ]:
#Word2Vec
# Word2Vec runs on tokenized sentences
X_train_tok= [nltk.word_tokenize(str(i)) for i in X_train]  
X_val_tok= [nltk.word_tokenize(str(i)) for i in X_val]

In [ ]:
from sklearn.model_selection import GridSearchCV

grid_param = [{
                'clf_LR__estimator__C': [100, 10, 1, 0.1, 0.01, 0.001, 0.0001],
                'clf_LR__estimator__penalty': ['l2'],
                'clf_LR__estimator__max_iter': [250],
               'clf_LR__estimator__solver' : ['newton-cg', 'lbfgs', ]
               }]

gridsearch = GridSearchCV(LR_pipeline_hyper_tuning, grid_param, cv=5, scoring='f1_weighted', verbose=0,n_jobs=-1) 
best_model = gridsearch.fit(X_train_tok,y_train_mdl_lbl_enc)

In [ ]:
best_model.best_params_

{'clf_LR__estimator__C': 100,
 'clf_LR__estimator__max_iter': 250,
 'clf_LR__estimator__penalty': 'l2',
 'clf_LR__estimator__solver': 'newton-cg'}

In [ ]:
best_model.score(X_val_tok,y_val_mdl_lbl_enc)

0.3000418919745818

In [ ]:
# Access the best set of parameters
best_params = gridsearch.best_params_
print(best_params)
# Stores the optimum model in best_pipe
best_LR_pipe = gridsearch.best_estimator_
print(best_LR_pipe)
 
result_df = pd.DataFrame.from_dict(gridsearch.cv_results_, orient='columns')
print(result_df.columns)

{'clf_LR__estimator__C': 100, 'clf_LR__estimator__max_iter': 250, 'clf_LR__estimator__penalty': 'l2', 'clf_LR__estimator__solver': 'newton-cg'}
Pipeline(steps=[('w2v',
                 <__main__.MeanEmbeddingVectorizer object at 0x7ff2e5d66710>),
                ('clf_LR',
                 OneVsRestClassifier(estimator=LogisticRegression(C=100,
                                                                  max_iter=250,
                                                                  solver='newton-cg')))])
Index(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time',
       'param_clf_LR__estimator__C', 'param_clf_LR__estimator__max_iter',
       'param_clf_LR__estimator__penalty', 'param_clf_LR__estimator__solver',
       'params', 'split0_test_score', 'split1_test_score', 'split2_test_score',
       'split3_test_score', 'split4_test_score', 'mean_test_score',
       'std_test_score', 'rank_test_score'],
      dtype='object')


In [ ]:
predictions = gridsearch.predict(X_val_tok)


In [ ]:
calc_metrics(y_val_mdl_lbl_enc,predictions)

Accuracy score:  0.47
precision_weighted: 0.75
recall_weighted: 0.47
f1_weighted: 0.3
Classification Report:
              precision    recall  f1-score   support

           0       0.47      1.00      0.64       994
           1       1.00      0.00      0.00         8
           2       1.00      0.00      0.00        35
           3       1.00      0.00      0.00         8
           4       1.00      0.00      0.00        64
           5       1.00      0.00      0.00        36
           6       1.00      0.00      0.00        30
           7       1.00      0.00      0.00        10
           8       1.00      0.00      0.00        21
           9       1.00      0.00      0.00        20
          10       1.00      0.00      0.00        22
          11       1.00      0.00      0.00        54
          12       1.00      0.00      0.00        60
          13       1.00      0.00      0.00         9
          14       1.00      0.00      0.00         7
          15       1.00   

**In the above Word2Vec model for Logistic regression, we could not find any embeddings trained, and the scores were below the benchmark models.**

In [ ]:
# End of the program
print("Model LR ; Vectorizer Word2Vec completed")